In [23]:
cd /home/jovyan

/home/jovyan


In [24]:
N_CORES = 128 # Где N_CORES - количество ядер CPU

import os

os.environ["MKL_NUM_THREADS"] = str(N_CORES)

os.environ["NUMEXPR_NUM_THREADS"] = str(N_CORES)

os.environ["OMP_NUM_THREADS"] = str(N_CORES)
from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline, KandinskyV22PriorEmb2EmbPipeline, KandinskyV22ControlnetImg2ImgPipeline, KandinskyV22Img2ImgPipeline
import torch
from PIL import Image
from PIL import ImageDraw, ImageFont
from copy import deepcopy
from deep_translator import GoogleTranslator

from rembg import remove
import sys
sys.path.append('/home/jovyan/cene655/ControlNet')
from annotator.util import resize_image, HWC3
from annotator.midas import MidasDetector
from diffusers import KandinskyV22PriorEmb2EmbPipeline
from diffusers import KandinskyV22ControlnetImg2ImgPipeline
import torch
import PIL
import torch
from diffusers.utils import load_image
from torchvision import transforms
from transformers import CLIPVisionModelWithProjection
from diffusers.models import UNet2DConditionModel
import numpy as np

In [25]:
def make_hint(img, detector):
    input_image = np.array(img)
    img = HWC3(input_image)#resize_image(, input_image.shape[1])
    print(img.shape)
    H, W, C = img.shape
    detected_map, _ = detector(img)
    detected_map = HWC3(detected_map)
    detected_map = torch.from_numpy(detected_map.copy()).float() / 255.0
    hint = detected_map.permute(2, 0, 1)
    return hint

In [26]:
detector = MidasDetector()

In [27]:
decoder = KandinskyV22Img2ImgPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-decoder', torch_dtype=torch.float16).to('cuda')

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
controlnet = KandinskyV22ControlnetImg2ImgPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-controlnet-depth', torch_dtype=torch.float16).to('cuda')

unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

In [29]:
prior = KandinskyV22PriorPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-prior', torch_dtype=torch.float16).to('cuda')

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [30]:
prompt = 'pizza in the space, photo'

In [31]:
768 + 64#704 1280

832

In [32]:
def prepare_caption(caption): return GoogleTranslator(source='ru', target='en').translate(caption)

In [33]:
def delete_bg(img, img2):
    min_size = min(img2.size[0], img2.size[1])

    img = img.resize((min_size, min_size))
    img = np.array(img)
    img2 = np.array(img2)
    output = remove(img)
    m = 0
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            m = max(output[i][j][3], m)
            if output[i][j][3] >= 80:
                img2[i][j] = output[i][j][:3]
    return Image.fromarray(img2)

In [34]:
def generate_avatar(decoder, controlnet, prior, prompt, text=None, style_image=None, face_image=None, animation=None):
        
    zero_img = PIL.Image.new(mode="RGB", size=(200, 200))
    img_emb = prior(prompt=prompt, num_inference_steps=50, num_images_per_prompt=1, guidance_scale=1.0).image_embeds
    if style_image is not None:
        img_emb2 = prior.interpolate(images_and_prompts=[style_image], weights=[1.0], num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0).image_embeds
        img_emb = img_emb * 0.55 + img_emb2 * 0.45
    negative_emb = prior(prompt='blow quility', num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0)
    images = decoder(image=zero_img, strength=1.0, image_embeds=img_emb, negative_image_embeds=negative_emb.negative_image_embeds, num_inference_steps=100, height=832, width=832, guidance_scale=4.0).images
    if face_image is not None:
        images[0] = delete_bg(face_image, images[0])
    if text is not None:
        img = deepcopy(images[0])
        W, H = img.size
        myFont = ImageFont.truetype('cene655/drawbench/DejaVuSansCondensed.ttf', 40)

        draw = ImageDraw.Draw(img)
        _, _, w, h = draw.textbbox((0, 0), text, font=myFont)
        draw.text(((W-w)/2, 25), text, font=myFont, fill=(255, 255, 255), stroke_width=2, stroke_fill=(0, 0, 0))
        hint = make_hint(img, detector).unsqueeze(0).half().to('cuda').repeat(1, 1, 1, 1)
        images2 = controlnet(image=img, strength=0.05, image_embeds=img_emb, negative_image_embeds=negative_emb.image_embeds, hint=hint, num_inference_steps=100, height=img.size[1], width=img.size[0])
        return images2[0][0]
    else:
        return images[0]

    

In [37]:
def generate_preview(decoder, contronet, prior, prompt, text=None, style_image=None, face_image=None, animation=None):
    prompt = '4k photo of ' + prompt
    zero_img = PIL.Image.new(mode="RGB", size=(200, 200))
    img_emb = prior(prompt=prompt, num_inference_steps=50, num_images_per_prompt=1, guidance_scale=1.5).image_embeds
    if style_image is not None:
        img_emb2 = prior.interpolate(images_and_prompts=[style_image], weights=[1.0], num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0).image_embeds
        img_emb = img_emb * 0.75 + img_emb2 * 0.25
    negative_emb = prior(prompt='blow quility', num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0)
    images = decoder(image=zero_img, strength=1.0, image_embeds=img_emb, negative_image_embeds=negative_emb.negative_image_embeds, num_inference_steps=250, height=704, width=1280, guidance_scale=4.0).images
    if face_image is not None:
        images[0] = delete_bg(face_image, images[0])
    if text is not None:
        img = deepcopy(images[0])
        W, H = img.size
        print(len(text))
        if len(text) < 52:
            myFont = ImageFont.truetype('cene655/drawbench/DejaVuSansCondensed.ttf', 50)
        else:
            myFont = ImageFont.truetype('cene655/drawbench/DejaVuSansCondensed.ttf', 35)
        draw = ImageDraw.Draw(img)
        _, _, w, h = draw.textbbox((0, 0), text, font=myFont)
        draw.text(((W-w)/2, 25), text, font=myFont, fill=(255, 255, 255), stroke_width=2, stroke_fill=(0, 0, 0))
        hint = make_hint(img, detector).unsqueeze(0).half().to('cuda').repeat(1, 1, 1, 1)
        #images2 = controlnet(image=img, strength=0.01, image_embeds=img_emb, negative_image_embeds=negative_emb.image_embeds, hint=hint, num_inference_steps=150, height=img.size[1], width=img.size[0])
        return img#images2[0][0]
    else:
        return images[0]

    

In [72]:
import imageio.v3 as iio
for i in range(93, len(df)):
    frame = Image.fromarray(iio.imread(
                'cene655/videos_c/rutube_hackathon_sochi/videos/' + df['video_name'].iloc[i],
                index=1000,
                plugin="pyav",
            ))
    out_img = generate_preview(decoder, controlnet, prior, prepare_caption(df['description'].iloc[i]), text=df['title'].iloc[i].replace('I', '\n').replace('|', '\n'), style_image=frame, face_image=None, animation=None)
    out_img.save(f'cene655/videos_c/previews1/{i}.jpg')


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

98
(704, 1280, 3)


...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

61
(704, 1280, 3)
